In [665]:
# This file takes the Data/Template folder and the files in it, and copies it in the Data folder but with some files altered:

# The aircraft.csv file is changes to alter the aircrafts


# The alt_aircraft.csv file is changed to alter the disruption of the aircrafts
# The flights.csv file is changed to alter the flights
# The rotations.csv file is changed to alter which aircrafts are used for which flights

In [666]:
import os
import csv
import random
import datetime
import shutil
import time
import re
from datetime import datetime, timedelta


# Make a copy of the Template folder in the Data folder with the name as input
data_folder = "Data/" + "01"

# If the folder already exists, delete it
if os.path.exists(data_folder):
    shutil.rmtree(data_folder)
shutil.copytree("Data/Template", data_folder)

# Data files
aircraft_file = data_folder + '/aircraft.csv'
airports_file = data_folder + '/airports.csv'
alt_aircraft_file = data_folder + '/alt_aircraft.csv'
alt_airports_file = data_folder + '/alt_airports.csv'
alt_flights_file = data_folder + '/alt_flights.csv'
config_file = data_folder + '/config.csv'
dist_file = data_folder + '/dist.csv'
flights_file = data_folder + '/flights.csv'
itineraries_file = data_folder + '/itineraries.csv'
positions_file = data_folder + '/position.csv'
rotations_file = data_folder + '/rotations.csv'


# 

In [667]:
# Function to read a CSV file, skip comment lines starting with '%', and stopping at lines starting with '#'
def read_csv_with_comments(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Filter out comment lines and stop at the ending line
    data_lines = []
    for line in lines:
        if line.startswith('#'):
            break
        if not line.startswith('%'):
            data_lines.append(line.strip())

    # Check if data_lines is empty
    if len(data_lines) == 0:
        return None  # Return None if the file has no data
    else:
        return data_lines


# Function to clear the file before adding new data
def clear_file(file_name):
    with open(file_name, 'w') as file:
        file.write('')  # Clear the file


In [668]:
def parse_time_with_day_offset(time_str, reference_date):
    if '+1' in time_str:
        time_str = time_str.replace('+1', '').strip()
        time_obj = datetime.strptime(time_str, '%H:%M')
        return datetime.combine(reference_date, time_obj.time()) + timedelta(days=1)
    else:
        return datetime.strptime(time_str, '%H:%M').replace(year=reference_date.year, month=reference_date.month, day=reference_date.day)


# Config

In [669]:
def parse_config(data_lines):
    config_dict = {}
    config_dict['RecoveryPeriod'] = {
        'StartDate': data_lines[0].split()[0],
        'StartTime': data_lines[0].split()[1],
        'EndDate': data_lines[0].split()[2],
        'EndTime': data_lines[0].split()[3]
    }

    def parse_costs(line):
        parts = re.split(r'\s+', line)
        costs = []
        for i in range(0, len(parts), 3):
            costs.append({'Cabin': parts[i], 'Type': parts[i+1], 'Cost': float(parts[i+2])})
        return costs

    config_dict['DelayCosts'] = parse_costs(data_lines[1])
    config_dict['CancellationCostsOutbound'] = parse_costs(data_lines[2])
    config_dict['CancellationCostsInbound'] = parse_costs(data_lines[3])

    def parse_downgrading_costs(line):
        parts = re.split(r'\s+', line)
        costs = []
        for i in range(0, len(parts), 4):
            costs.append({'FromCabin': parts[i], 'ToCabin': parts[i+1], 'Type': parts[i+2], 'Cost': float(parts[i+3])})
        return costs

    config_dict['DowngradingCosts'] = parse_downgrading_costs(data_lines[4])
    config_dict['PenaltyCosts'] = [float(x) for x in re.split(r'\s+', data_lines[5])]
    config_dict['Weights'] = [float(x) for x in re.split(r'\s+', data_lines[6])]
    return config_dict

# Read the config file and parse the data
config_lines = read_csv_with_comments(config_file)
if config_lines:
    config_dict = parse_config(config_lines)
    print(config_dict)
else:
    # give error
    print('Config file is empty')

# start date is random day in september 2024. strip only dd/mm/yy
recovery_start_date = datetime(2024, 9, random.randint(1, 30)).strftime('%d/%m/%y')
# start time is either 05:00 or 06:00 or 07:00
recovery_start_time = f'{random.randint(5, 7)}:00'
# end date is same as start date
recovery_end_date = recovery_start_date
# end time is random half-hour interval between 20:00 and 23:30
recovery_end_time = f'{random.randint(20, 23)}:{random.choice(["00", "30"])}'

print(recovery_start_date, recovery_start_time, recovery_end_date, recovery_end_time)

# the rest is as in the template

# Regenerate config file with new data
def generate_config_file(file_name):
    # Clear the file before writing new data
    clear_file(file_name)
    
    # Write the new data to the file
    with open(file_name, 'w') as file:
        file.write('%RecoveryPeriod\n')
        file.write(f"{recovery_start_date} {recovery_start_time} {recovery_end_date} {recovery_end_time}\n")
        
        # adjust the cofig_dict
        config_dict['RecoveryPeriod'] = {   
            'StartDate': recovery_start_date,
            'StartTime': recovery_start_time,
            'EndDate': recovery_end_date,
            'EndTime': recovery_end_time
        }

        file.write('%DelayCosts\n')
        for cost in config_dict['DelayCosts']:
            file.write(f"{cost['Cabin']} {cost['Type']} {cost['Cost']} ")
        
        file.write('\n%CancellationCostsOutbound\n')
        for cost in config_dict['CancellationCostsOutbound']:
            file.write(f"{cost['Cabin']} {cost['Type']} {cost['Cost']} ")
        
        file.write('\n%CancellationCostsInbound\n')
        for cost in config_dict['CancellationCostsInbound']:
            file.write(f"{cost['Cabin']} {cost['Type']} {cost['Cost']} ")
        
        file.write('\n%DowngradingCosts\n')
        for cost in config_dict['DowngradingCosts']:
            file.write(f"{cost['FromCabin']} {cost['ToCabin']} {cost['Type']} {cost['Cost']} ")
        
        file.write('\n%PenaltyCosts\n')
        for cost in config_dict['PenaltyCosts']:
            file.write(f"{cost} ")
        file.write('\n')
        
        file.write('%Weights\n')
        for weight in config_dict['Weights']:
            file.write(f"{weight} ")
        file.write('\n')
        
        file.write('#')

from datetime import datetime


# Generate the new config file
generate_config_file(config_file)

# Parse date and time separately
start_date = datetime.strptime(config_dict['RecoveryPeriod']['StartDate'], '%d/%m/%y')
start_time = datetime.strptime(config_dict['RecoveryPeriod']['StartTime'], '%H:%M')
end_date = datetime.strptime(config_dict['RecoveryPeriod']['EndDate'], '%d/%m/%y')
end_time = datetime.strptime(config_dict['RecoveryPeriod']['EndTime'], '%H:%M')

# Combine date and time into a single datetime object
start_datetime = start_date.replace(hour=start_time.hour, minute=start_time.minute)
end_datetime = end_date.replace(hour=end_time.hour, minute=end_time.minute)

# Print the combined datetime
print(f"Start datetime: {start_datetime}")


{'RecoveryPeriod': {'StartDate': '10/01/08', 'StartTime': '06:00', 'EndDate': '10/01/08', 'EndTime': '23:30'}, 'DelayCosts': [{'Cabin': 'F', 'Type': 'D', 'Cost': 0.75}, {'Cabin': 'F', 'Type': 'C', 'Cost': 1.08}, {'Cabin': 'F', 'Type': 'I', 'Cost': 1.5}, {'Cabin': 'B', 'Type': 'D', 'Cost': 0.5}, {'Cabin': 'B', 'Type': 'C', 'Cost': 0.75}, {'Cabin': 'B', 'Type': 'I', 'Cost': 1.0}, {'Cabin': 'E', 'Type': 'D', 'Cost': 0.33}, {'Cabin': 'E', 'Type': 'C', 'Cost': 0.5}, {'Cabin': 'E', 'Type': 'I', 'Cost': 0.66}], 'CancellationCostsOutbound': [{'Cabin': 'F', 'Type': 'D', 'Cost': 1000.0}, {'Cabin': 'F', 'Type': 'C', 'Cost': 2000.0}, {'Cabin': 'F', 'Type': 'I', 'Cost': 3000.0}, {'Cabin': 'B', 'Type': 'D', 'Cost': 650.0}, {'Cabin': 'B', 'Type': 'C', 'Cost': 1300.0}, {'Cabin': 'B', 'Type': 'I', 'Cost': 2000.0}, {'Cabin': 'E', 'Type': 'D', 'Cost': 350.0}, {'Cabin': 'E', 'Type': 'C', 'Cost': 650.0}, {'Cabin': 'E', 'Type': 'I', 'Cost': 1000.0}], 'CancellationCostsInbound': [{'Cabin': 'F', 'Type': 'D', 

# Aircraft

In [670]:
import os
import random

# Predefined values based on the document
aircraft_types = [
    {'Model': 'B737', 'Family': 'Boeing', 'Config': '10/20/160', 'Dist': 5000, 'Cost/h': 3000.0, 'TurnRound': 75, 'Transit': 50, 'Orig': ['CDG'], 'Maint': ['NULL']},
    {'Model': 'A320', 'Family': 'Airbus', 'Config': '0/0/123', 'Dist': 5000, 'Cost/h': 1800.0, 'TurnRound': 30, 'Transit': 30, 'Orig': ['CFE'], 'Maint': ['NULL']}
]

# Define the range of the total number of aircraft to generate
total_aircraft_range = (5, 10)


aircraft_ids = []  # List to hold the generated aircraft IDs


# Function to generate aircraft.csv
def generate_aircraft_file(file_name):
    # Clear the aircraft file before writing new data
    clear_file(file_name)
    
    # Determine the total number of aircraft to generate
    total_aircraft = random.randint(total_aircraft_range[0], total_aircraft_range[1])
    print(f"Generating {total_aircraft} aircraft data")
    
    aircraft_data = []  # List to hold the aircraft details before writing

    aircraft_counter = {aircraft['Model']: 0 for aircraft in aircraft_types}  # Track the count per type

    # Loop until we have the required number of aircraft
    for _ in range(total_aircraft):
        # Randomly choose an aircraft type
        aircraft_type = random.choice(aircraft_types)
        model = aircraft_type['Model']
        family = aircraft_type['Family']
        config = aircraft_type['Config']
        dist = aircraft_type['Dist']
        cost_h = aircraft_type['Cost/h']
        turn_round = aircraft_type['TurnRound']
        transit = aircraft_type['Transit']
        origins = aircraft_type['Orig']
        maintenances = aircraft_type['Maint']
        
        # Increment the aircraft counter for the chosen type
        aircraft_counter[model] += 1
        aircraft_id = f"{model}#{aircraft_counter[model]}"  # Create an incremental aircraft ID
        
        # Add the aircraft ID to the list
        aircraft_ids.append(aircraft_id)
        
        orig = random.choice(origins)
        maint = random.choice(maintenances)
        
        # Add the aircraft details to the list
        aircraft_data.append(f"{aircraft_id} {model} {family} {config} {dist} {cost_h} {turn_round} {transit} {orig} {maint}")
    
    # Sort the aircraft data alphabetically based on the aircraft ID
    aircraft_data.sort()

    

    # Write the sorted data to the file
    with open(file_name, 'w') as file:
        # Write the comment with %
        file.write('%Aircraft Model Family Config Dist Cost/h TurnRound Transit Orig Maint\n')
        
        # Write each sorted aircraft line
        for aircraft in aircraft_data:
            file.write(f"{aircraft}\n")
        
        # End with the '#'
        file.write('#')

# Example usage
generate_aircraft_file(aircraft_file)

print(f"Aircraft data has been generated successfully in {aircraft_file}")


Generating 5 aircraft data
Aircraft data has been generated successfully in Data/01/aircraft.csv


# alt_aircraft

In [671]:
# now the alt_aircraft_file
# format is:
"""
%Aircraft StartDate StartTime EndDate EndTime
B767#3 10/01/08 06:00 10/01/08 16:00
#
"""
# the disrupted aircraft are randomly chosen from all aircrafts from list of aircraft_ids
# the number of disrupted aircraft is set 



# Define how many aircrafts to disrupt, cannot be more than the total number of aircrafts
amount_aircraft_disrupted = 1
amount_aircraft_disrupted = min(amount_aircraft_disrupted, len(aircraft_ids))
min_delta_start_unavailability = 0
max_delta_start_unavailability = 120

min_period_unavailability = 120
max_period_unavailability = 720

# Function to generate alt_aircraft.csv
def generate_alt_aircraft_file(file_name):
    # Clear the file before writing new data
    clear_file(file_name)
    
    # Pick random aircraft to disrupt, as many as total_aircraft_disrupted
    disrupted_aircraft_id = random.sample(aircraft_ids, amount_aircraft_disrupted)

    disrupted_aircraft_data = []  # List to hold the disrupted aircraft details before writing

    # Loop until we have the required number of disrupted aircraft
    for aircraft_id in disrupted_aircraft_id:
        # start date is same as config_dict RecoveryPeriod start date. use parse function
        start_date = config_dict['RecoveryPeriod']['StartDate']
        print(f"Start date: {start_date}")

        # Start time of unavailability is same as config_dict RecoveryPeriod start time plus random number of minutes between 0 and 120
        start_time_recovery = config_dict['RecoveryPeriod']['StartTime']
        start_unavail = datetime.strptime(start_time_recovery, '%H:%M')
        start_unavail += timedelta(minutes=random.randint(min_delta_start_unavailability, max_delta_start_unavailability))
        start_unavail = start_unavail.strftime('%H:%M')
        print(f"Start unavail: {start_unavail}")

        # End date is same as config_dict RecoveryPeriod end date
        end_date = config_dict['RecoveryPeriod']['EndDate']
        print(f"End date: {end_date}")

        # End time is the start_time plus random number of minutes between 120 and 720

        # start_unavail is in format HH:MM. first convert it to datetime object
        start_unavail_parse = datetime.strptime(start_unavail, '%H:%M')
        end_unavail = start_unavail_parse + timedelta(minutes=random.randint(min_period_unavailability, max_period_unavailability))
        end_unavail = end_unavail.strftime('%H:%M')
        print(f"End unavail: {end_unavail}")

        print(start_date)
        print(start_unavail)
        # Add the disrupted aircraft details to the list
        disrupted_aircraft_data.append(f"{aircraft_id} {start_date} {start_unavail} {end_date} {end_unavail}")

        

    # Write the disrupted aircraft data to the file
    with open(file_name, 'w') as file:
        # Write the comment with %
        file.write('%Aircraft StartDate StartTime EndDate EndTime\n')
        
        # Write each disrupted aircraft line
        for aircraft in disrupted_aircraft_data:
            file.write(f"{aircraft}\n")
        
        # End with the '#'
        file.write('#')
    
    
    # Add the disrupted aircraft details to the list wit format:
    # Aircraft_id StartDate StartTime EndDate EndTime

generate_alt_aircraft_file(alt_aircraft_file)

print(f"Disrupted aircraft data has been generated successfully in {alt_aircraft_file}")


Start date: 02/09/24
Start unavail: 05:14
End date: 02/09/24
End unavail: 09:32
02/09/24
05:14
Disrupted aircraft data has been generated successfully in Data/01/alt_aircraft.csv


# Airports

In [672]:
# Just a random set of airports now
airports = ['AMS', 'CDG', 'FRA', 'LHR', 'MAD', 'MUC', 'ORY', 'TXL', 'VIE', 'ZRH', 'BCN', 'BRU', 'CPH', 'DUB', 'IST']

# Flights

In [673]:
# Function to convert flights file lines to a dictionary with corresponding values
def parse_flights(data_lines):
    flights_dict = {}
    for line in data_lines:
        parts = re.split(r'\s+', line)
        flight = int(parts[0])
        flights_dict[flight] = {
            'Orig': parts[1],
            'Dest': parts[2],
            'DepTime': parts[3],
            'ArrTime': parts[4],
            'PrevFlight': int(parts[5])
        }
    return flights_dict


flights_lines = read_csv_with_comments(flights_file)
if flights_lines:
    flights_dict = parse_flights(flights_lines)
    # print(flights_dict)
else:
    flights_dict = None


# print(flights_dict)

# Creating a new flights.csv file with new data

# Average number of flights per aircraft
average_flights_per_aircraft = 4
std_dev_flights_per_aircraft = 1
total_flights = len(aircraft_ids) * average_flights_per_aircraft

flight_rotation_data = {}

# Regenerate flights.csv file with new data
def generate_flights_file(file_name):
    # Clear the file before writing new data
    clear_file(file_name)
    
    # Data format is Flight Orig Dest DepTime ArrTime PrevFlight
    flights_dict = {}
    # format of the flights_dict is {1: {'Orig': 'BKK', 'Dest': 'CDG', 'DepTime': '18:00', 'ArrTime': '01:15+1', 'PrevFlight': 0}, 2: {'Orig': 'NCE', 'Dest': 'CDG', 'DepTime': '08:00', 'ArrTime': '09:30', 'PrevFlight': 0}, 3: {'Orig': 'CDG', 'Dest': 'LHR', 'DepTime': '6:20', 'ArrTime': '07:20', 'PrevFlight': 0}, 4: {'Orig': 'CDG', 'Dest': 'LHR', 'DepTime': '08:30', 'ArrTime': '10:30', 'PrevFlight': 0}, 5: {'Orig': 'LHR', 'Dest': 'JFK', 'DepTime': '12:00', 'ArrTime': '15:30', 'PrevFlight': 0}, 6: {'Orig': 'NCE', 'Dest': 'LHR', 'DepTime': '07:30', 'ArrTime': '11:15', 'PrevFlight': 0}, 7: {'Orig': 'CDG', 'Dest': 'LHR', 'DepTime': '6:00', 'ArrTime': '07:20', 'PrevFlight': 0}, 8: {'Orig': 'LHR', 'Dest': 'CDG', 'DepTime': '12:30', 'ArrTime': '13:30', 'PrevFlight': 0}, 9: {'Orig': 'CDG', 'Dest': 'NCE', 'DepTime': '14:35', 'ArrTime': '16:05', 'PrevFlight': 0}, 10: {'Orig': 'LHR', 'Dest': 'NCE', 'DepTime': '13:10', 'ArrTime': '14:55', 'PrevFlight': 0}, 11: {'Orig': 'JFK', 'Dest': 'LHR', 'DepTime': '16:30', 'ArrTime': '19:00', 'PrevFlight': 0}, 12: {'Orig': 'LHR', 'Dest': 'CDG', 'DepTime': '07:00', 'ArrTime': '08:15', 'PrevFlight': 0}, 13: {'Orig': 'LHR', 'Dest': 'NCE', 'DepTime': '13:10', 'ArrTime': '14:55', 'PrevFlight': 0}, 14: {'Orig': 'JFK', 'Dest': 'LHR', 'DepTime': '09:00', 'ArrTime': '14:00', 'PrevFlight': 0}, 15: {'Orig': 'LHR', 'Dest': 'CDG', 'DepTime': '07:15', 'ArrTime': '08:15', 'PrevFlight': 0}, 16: {'Orig': 'LHR', 'Dest': 'CDG', 'DepTime': '09:00', 'ArrTime': '11:05', 'PrevFlight': 0}}



    # generate a number for each aircraft how many flights, based on average_flights_per_aircraft
    # amount_flights_per_aircraft is like {"B737#1": 3, "B737#2": 4, "A320#1": 5}
    amount_flights_per_aircraft = {}
    
    flights_left_to_generate = total_flights
    for aircraft_id in aircraft_ids:
        if flights_left_to_generate <= 0:
            amount_flights_per_aircraft[aircraft_id] = 0  # Assign 0 if no flights are left to assign
            continue
        amount_flights_per_aircraft[aircraft_id] = min(
            random.randint(average_flights_per_aircraft - std_dev_flights_per_aircraft, 
                        average_flights_per_aircraft + std_dev_flights_per_aircraft),
            flights_left_to_generate
        )
        flights_left_to_generate -= amount_flights_per_aircraft[aircraft_id]

        # amount_flights_per_aircraft.append(random.randint(average_flights_per_aircraft - std_dev_flights_per_aircraft, average_flights_per_aircraft + std_dev_flights_per_aircraft))
        # assigned_flights = sum(amount_flights_per_aircraft)
    
    flight_id = 0  
    flights_per_aircraft = {}


    # Generate random flights with Flight Orig Dest DepTime ArrTime PrevFlight
    # We go through each aircraft and generate the amount of flights for that aircraft
    for aircraft_id in aircraft_ids:
        # while the assigned flights do not exceed the amount of flights for this aircraft
        flight_id_ac_specific = 0

        # for i in range(amount_flights_per_aircraft[aircraft_id]) if amount_flights_per_aircraft[aircraft_id] > 0 else 1
        for i in range(amount_flights_per_aircraft[aircraft_id]):
            flight_id += 1
            flight_id_ac_specific += 1

            # Randomly choose an origin and destination
            orig = random.choice(airports)
            dest = random.choice(airports)
            while orig == dest:
                dest = random.choice(airports)

            # The first flight for each aircraft starts between 6:00 and 12:00
            # The rest of the flights for the aircraft are scheduled after the previous flight
            if flight_id_ac_specific == 1:
                # departure time in real datetime format
                dep_time = f"{random.randint(6, 12)}:{random.choice(['00', '15', '30', '45'])}"
                dep_time_obj = parse_time_with_day_offset(dep_time, start_datetime)
                # print(f"Dep time: {dep_time_obj}")
                # arriaval time is departure time plus random number of hours and minutes between 1 and 6
                arr_time_obj = parse_time_with_day_offset(dep_time, start_datetime) + timedelta(hours=random.randint(1, 4), minutes=random.randint(0, 59))
                # print(f"Arr time: {arr_time_obj}")

                

                # if the arrival time is on the day after the start of the recovery period, add +1 to the time
            


                if arr_time_obj.day > start_datetime.day:
                    arr_time = f"{arr_time_obj.strftime('%H:%M')}+1"
                else:
                    arr_time = arr_time_obj.strftime('%H:%M')


                # print(f"First flight for {aircraft_id} from {orig} to {dest} at {dep_time} and arrival at {arr_time}")
                prev_flight = 0

                # save to the flights_dict
                flights_dict[flight_id] = {'Orig': orig, 'Dest': dest, 'DepTime': dep_time, 'ArrTime': arr_time, 'PrevFlight': prev_flight, 'Aircraft': aircraft_id}

                # save to flight_rotation_data: id, date (dd/mm/yy), aircraft
                flight_rotation_data[flight_id] = {'Aircraft': aircraft_id}

                
                
            else:
                # for the second, third, etc. flight for the aircraft, the departure time is the arrival time of the previous flight plus random number of hours and minutes between 1 and 6
                
                # get the arrival time of the previous flight of the same aircraft
                arr_time_prev = flights_dict[flight_id - 1]['ArrTime']
                arr_time_prev_obj = parse_time_with_day_offset(arr_time_prev, datetime.strptime(config_dict['RecoveryPeriod']['StartDate'], '%d/%m/%y'))

                dep_time_obj = arr_time_prev_obj + timedelta(hours=random.randint(0, 2), minutes=random.randint(0, 59))


                # arrival time is departure time plus random number of hours and minutes between 1 and 6
                arr_time_obj = dep_time_obj + timedelta(hours=random.randint(1, 4), minutes=random.randint(0, 59))

                start_date = datetime.strptime(config_dict['RecoveryPeriod']['StartDate'], '%d/%m/%y')

                # if the arrival time is on the day after the start of the recovery period, add +1 to the time
                if arr_time_obj.day > start_date.day:
                    arr_time = f"{arr_time_obj.strftime('%H:%M')}+1"
                else:
                    arr_time = arr_time_obj.strftime('%H:%M')

                # if the departure time is on the day after the start of the recovery period, add +1 to the time
                if dep_time_obj.day > start_date.day:
                    dep_time = f"{dep_time_obj.strftime('%H:%M')}+1"
                else:
                    dep_time = dep_time_obj.strftime('%H:%M')


                # print(f"Flight for {aircraft_id} from {orig} to {dest} at {dep_time} and arrival at {arr_time}")
                prev_flight = 0

                # save to the flights_dict
                flights_dict[flight_id] = {'Orig': orig, 'Dest': dest, 'DepTime': dep_time, 'ArrTime': arr_time, 'PrevFlight': prev_flight, 'Aircraft': aircraft_id}

                # save to flight_rotation_data
                flight_rotation_data[flight_id] = {'Aircraft': aircraft_id}


                # stop the loop for this aircraft if the arrival time is after the end of the recovery period
                if arr_time_obj > end_datetime:
                    break

        


    # Write the new flights data to the file
    with open(file_name, 'w') as file:
        # Write the comment with %
        file.write('%Flight Orig Dest DepTime ArrTime PrevFlight\n')
        
        # Write each flight line
        for flight_id, flight_data in flights_dict.items():
            file.write(f"{flight_id} {flight_data['Orig']} {flight_data['Dest']} {flight_data['DepTime']} {flight_data['ArrTime']} {flight_data['PrevFlight']}\n")
            # print(f"{flight_id} {flight_data['Orig']} {flight_data['Dest']} {flight_data['DepTime']} {flight_data['ArrTime']} {flight_data['PrevFlight']} {flight_data['Aircraft']}")

        # End with the '#'
        file.write('#')

# Example usage
generate_flights_file(flights_file)

print(f"Flights data has been generated successfully in {flights_file}")


Flights data has been generated successfully in Data/01/flights.csv


# Rotations

In [674]:
# now the rotations file
# format is:
"""
%Flight DepDate Aircraft
1 10/01/08 B767#2
7 10/01/08 B767#1
...
#
"""
# All the info of the flights is in the flight_rotation_data dictionary
# We need to generate the rotations based on the flights

# Function to generate rotations.csv
def generate_rotations_file(file_name):
    # Clear the file before writing new data
    clear_file(file_name)
    
    # Data format is Flight DepDate Aircraft
    rotations_data = []
    
    # Go through the flight_rotation_data and generate the rotations
    for flight_id, flight_data in flight_rotation_data.items():
        # Get the aircraft for the flight
        aircraft = flight_data['Aircraft']
        # Get the date of the flight
        dep_date = start_datetime.strftime('%d/%m/%y')
        
        # Add the rotation data to the list
        rotations_data.append(f"{flight_id} {dep_date} {aircraft}")

    # Write the rotations data to the file
    with open(file_name, 'w') as file:
        # Write the comment with %
        file.write('%Flight DepDate Aircraft\n')
        
        # Write each rotation line
        for rotation in rotations_data:
            file.write(f"{rotation}\n")
        
        # End with the '#'
        file.write('#')

# Example usage
generate_rotations_file(rotations_file)

print(f"Rotations data has been generated successfully in {rotations_file}")

Rotations data has been generated successfully in Data/01/rotations.csv
